In [18]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time, csv
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.relative_locator import locate_with

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())

In [3]:
page_url = 'https://www.novibet.gr/en/sports'
driver.get(page_url)
cookies = WebDriverWait(driver,5).until(EC.element_to_be_clickable((By.CLASS_NAME, 'acceptCookies_button'))).click()

In [4]:
# Close Log in pop up window
x_button = driver.find_element(By.CSS_SELECTOR, '[data-cy="closeBtn"]').click()

In [6]:
# Click Daily Coupon
dayly_coupon_button = driver.find_element(By.CSS_SELECTOR, 'a.ng-star-inserted[title="Daily coupon"]').click()

In [ ]:
# Wait for the dailyCoupon_body element to be present on the page
wait = WebDriverWait(driver, 10)
# By deafult is the football first
daily_coupon_body = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'dailyCoupon_body')))

# Extract footbal text
football_string = daily_coupon_body.text

In [ ]:
football_string = daily_coupon_text_football

In [260]:
# Create list from the initial string
initial_list = football_string.split('\n')

# Remove first elements of the list not needed
remove_elements = ['Daily coupon','Football','Tennis','Basketball',
                   '24 hours','12 hours','3 hours','Popular First']

football_list = [x for x in initial_list if x not in remove_elements]


# Use padding for 'Markets are not available'
for i in range(len(football_list)):
    if football_list[i] == 'Markets are not available':
        football_list.insert(i+1, 'No_market')
        football_list.insert(i+2, 'No_market')
        football_list.insert(i+3, 'No_market')

In [261]:
# Create sublists based on Championship
championship = [x for x in football_list if ' - ' in x]
index_championship = [i for i,x in enumerate(football_list) if ' - ' in x]
sublists_championships = [football_list[i:j] for i, j in zip([0]+index_championship, index_championship + [len(football_list)])]
# Exclude the initial empty list from sublist_championships
sublists_championships = sublists_championships[1:]

In [281]:
# Create dataframe 
df_football = pd.DataFrame(columns = ['championship','team1','team2','time','1','odds_1','X','odds_X','2','odds_2',
                             'Over','odds_over','Under','odds_under','GG','odds_gg','NG','odds_ng'])

# Loop through all championships
for j in range(len(sublists_championships)):
     
    team1_lst, team2_lst = [],[]
    time_lst = []
    one_lst, one_odds_lst, x_lst, x_odds_lst, two_lst, two_odds_lst = [],[],[],[],[],[]
    over_lst, over_odds_lst, under_lst, under_odds_lst = [],[],[],[]
    gg_lst, gg_odds_lst, ng_lst, ng_odds_lst = [],[],[],[]

    # Create list with only teams and odds / exclude the championship at the beginning
    teams_only_lst = sublists_championships[j][1:]
    championship_lst = [sublists_championships[j][0] for _ in range(len(teams_only_lst) // 17)]

    i = 0
    while i < len(teams_only_lst):
        team1_lst.append(teams_only_lst[i])
        team2_lst.append(teams_only_lst[i+1])
        time_lst.append(teams_only_lst[i+2])
        one_lst.append(teams_only_lst[i+3])
        one_odds_lst.append(teams_only_lst[i+4])
        x_lst.append(teams_only_lst[i+5])
        x_odds_lst.append(teams_only_lst[i+6])
        two_lst.append(teams_only_lst[i+7])
        two_odds_lst.append(teams_only_lst[i+8])
        over_lst.append(teams_only_lst[i+9])
        over_odds_lst.append(teams_only_lst[i+10])
        under_lst.append(teams_only_lst[i+11])
        under_odds_lst.append(teams_only_lst[i+12])
        gg_lst.append(teams_only_lst[i+13])
        gg_odds_lst.append(teams_only_lst[i+14])
        ng_lst.append(teams_only_lst[i+15])
        ng_odds_lst.append(teams_only_lst[i+16])
        i = i + 17

    # Create the dataframe
    df = pd.DataFrame({
        'championship': championship_lst,
        'team1': team1_lst,
        'team2': team2_lst,
        'time': time_lst,
        '1': one_lst,
        'odds_1': one_odds_lst,
        'X': x_lst,
        'odds_X': x_odds_lst,
        '2': two_lst,
        'odds_2': two_odds_lst,
        'Over': over_lst,
        'odds_over': over_odds_lst,
        'Under': under_lst,
        'odds_under': under_odds_lst,
        'GG': gg_lst,
        'odds_gg': gg_odds_lst,
        'NG': ng_lst,
        'odds_ng': ng_odds_lst
    })

    df_football = df_football.append(df, ignore_index = True)

In [283]:
df_football

,championship,team1,team2,time,1,odds_1,X,odds_X,2,odds_2,Over,odds_over,Under,odds_under,GG,odds_gg,NG,odds_ng
0,Argentina - Liga Profesional,Velez Sarsfield,Central Cordoba Santiago,Wed 02:00,1,1.74,X,3.55,2,5.20,O 2.5,2.35,U 2.5,1.59,GG,2.25,NG,1.63
1,Africa - Africa Cup of Nations - Qualification,Benin,Rwanda,Wed 17:00,1,1.74,X,3.45,2,4.85,O 2.5,2.25,U 2.5,1.63,GG,2.10,NG,1.68
2,Africa - Africa Cup of Nations - Qualification,Sierra Leone,Sao Tome and Principe,Wed 19:00,1,1.10,X,7.80,2,18.50,O 3.5,1.79,U 3.5,1.96,Markets are not available,No_market,No_market,No_market
3,England - Isthmian Premier Division,Lewes,Canvey Island,21:45,1,2.15,X,3.45,2,2.95,O 2.5,1.87,U 2.5,1.83,GG,1.74,NG,1.97
4,England - Isthmian Premier Division,Potters Bar Town FC,Bognor Regis Town,21:45,1,2.70,X,3.45,2,2.25,O 2.5,1.67,U 2.5,2.08,GG,1.57,NG,2.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,England - National League South,Chippenham,Weymouth,21:45,1,2.30,X,3.30,2,3.05,O 2.5,2.04,U 2.5,1.76,GG,1.81,NG,1.94
105,England - National League South,Farnborough,Slough,21:45,1,1.61,X,3.90,2,5.30,O 2.5,2.10,U 2.5,1.70,GG,2.15,NG,1.66
106,England - National League South,St. Albans,Cheshunt,21:45,1,1.49,X,4.40,2,5.90,O 2.5,1.79,U 2.5,1.99,GG,1.94,NG,1.80
107,England - National League South,Welling,Taunton Town FC,21:45,1,2.15,X,3.30,2,3.30,O 2.5,2.20,U 2.5,1.65,GG,1.93,NG,1.81


In [201]:
# Create the dataframe
df = pd.DataFrame({
    'Championship': championship_lst,
    'team1': team1_lst,
    'team2': team2_lst,
    'time': time_lst,
    '1': one_lst,
    'odds_1': one_odds_lst,
    'X': x_lst,
    'odds_X': x_odds_lst,
    '2': two_lst,
    'odds_2': two_odds_lst,
    'Over': over_lst,
    'odds_over': over_odds_lst,
    'Under': under_lst,
    'odds_under': under_odds_lst,
    'GG': gg_lst,
    'odds_gg': gg_odds_lst,
    'NG': ng_lst,
    'odds_ng': ng_odds_lst
})

In [194]:
df

,Championship,team1,team2,time,1,odds_1,X,odds_X,2,odds_2,Over,odds_over,Under,odds_under,GG,odds_gg,NG,odds_ng
0,Africa - Africa Cup of Nations - Qualification,Benin,Rwanda,Wed 17:00,1,1.74,X,3.45,2,4.85,O 2.5,2.25,U 2.5,1.63,GG,2.10,NG,1.68
1,Africa - Africa Cup of Nations - Qualification,Sierra Leone,Sao Tome and Principe,Wed 19:00,1,1.10,X,7.80,2,18.50,O 3.5,1.79,U 3.5,1.96,Markets are not available,No_market,No_market,No_market


In [175]:
time_lst

['Wed 17:00', 'Wed 19:00']

In [150]:
team1_lst

[[],
 ['Scotland - League 2',
  'Albion Rovers',
  'Stranraer',
  '21:45',
  '1',
  '2.30',
  'X',
  '3.35',
  '2',
  '2.95',
  'O 2.5',
  '1.93',
  'U 2.5',
  '1.84',
  'GG',
  '1.73',
  'NG',
  '2.03',
  'Stirling Albion',
  'Forfar',
  '21:45',
  '1',
  '1.74',
  'X',
  '3.65',
  '2',
  '4.55',
  'O 2.5',
  '1.91',
  'U 2.5',
  '1.87',
  'GG',
  '1.84',
  'NG',
  '1.90'],
 ['Algeria - Cup',
  'US Chaouia',
  'CR Belouizdad',
  'Wed 15:00',
  '1',
  '11.25',
  'X',
  '5.80',
  '2',
  '1.21',
  'O 2.5',
  '1.59',
  'U 2.5',
  '2.25',
  'GG',
  '2.25',
  'NG',
  '1.60']]

In [15]:
import re

championship_pattern = r'(?<=Popular First\n).*?(?=\n)'
teams_pattern = r'(?<=\n)[A-Za-z\s]+\n[A-Za-z\s]+\n'
time_pattern = r'\b(?:Mon|Tue|Wed|Thu|Fri|Sat|Sun)[\s\d]{1,2}:[\d]{2}\b'
bet_categories_pattern = r'(?:1|X|2|O[\s\d\.]+|U[\s\d\.]+|GG|NG)(?=\n)'

# extract the information using regex
championship = re.search(championship_pattern, daily_coupon_text).group().strip()
teams = re.findall(teams_pattern, daily_coupon_text)
time = re.findall(time_pattern, daily_coupon_text)
bet_categories = re.findall(bet_categories_pattern, daily_coupon_text)

# extract the odds
odds_pattern = r'[\d]+\.[\d]+'
odds = []
for i in range(len(teams)):
    odds.append(re.findall(odds_pattern, daily_coupon_text.split(teams[i])[1])[0:7])

In [17]:
teams

['Football\nBasketball\nTennis\n',
 'Velez Sarsfield\nCentral Cordoba Santiago\n',
 'Benin\nRwanda\n',
 'Sierra Leone\nSao Tome and Principe\n',
 'Lewes\nCanvey Island\n',
 'Potters Bar Town FC\nBognor Regis Town\n',
 'Accrington\nPlymouth\n',
 'Barnsley\nSheffield Wed\n',
 'Banbury\nCurzon Ashton FC\n',
 'Farsley Celtic FC\nDarlington\n',
 'Kings Lynn Town\nAFC Fylde\n',
 'Southport\nAlfreton Town FC\n',
 'Spennymoor Town\nLeamington\n',
 'Gainsborough\nMarine\n',
 'Marske United FC\nLiversedge\n',
 'Alvechurch\nBarwell\n',
 'Stratford Town\nLeiston FC\n',
 'Bradford\nCarlisle\n',
 'Crawley Town\nDoncaster\n',
 'Gillingham\nCrewe\n',
 'Chesham United FC\nPlymouth Parkway\n',
 'Hanwell Town\nPoole Town\n',
 'Harrow Borough\nBracknell Town\n',
 'Weiden\nSV Donaustauf\n',
 'Markets are not available\nGermania Ratingen\n',
 'MSV Dusseldorf\nKray\n',
 'Oldenburger SV\nFC Dornbreite\n',
 'SC Union Nettetal\nTSV Meerbusch\n',
 'Sportfreunde Baumberg\nVfB Homberg\n',
 'SV Sonsbeck\nKleve\n',
